with adapter + t5 decoder

### mount gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
os.chdir('/content/drive/MyDrive/rna-binding')

In [ ]:
!pip install transformers

### data prep

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('final_attract_db_with_emb.csv')

In [ ]:
import pickle
# Load the dictionary back from the pickle file.
with open("rbp_seqs_dict.pkl", "rb") as f:
    rbp_seqs_dict = pickle.load(f)

In [ ]:
import numpy as np
rna_motif_emb = np.load('rna_motif_emb.npy', allow_pickle=True)

In [ ]:
len(rna_motif_emb[1])

7

In [ ]:
data = data.drop(columns=['rna_motif_emb', 'rbp_esm_emb'])

In [ ]:
data['rna_motif_emb'] = rna_motif_emb

In [ ]:
data['rbp_esm_emb'] = data['RBP_sequence'].map(rbp_seqs_dict)

In [ ]:
# Convert list of tensors to numpy array
def tensors_to_numpy(tensor_list):
    return np.stack([t.numpy() for t in tensor_list])

# Apply the conversion to the 'rbp_esm_emb' column
data['rbp_esm_emb'] = data['rbp_esm_emb'].apply(tensors_to_numpy)

In [ ]:
data

,Gene_name,Gene_id,Motif,RBP_sequence,rna_motif_emb,rbp_esm_emb
0,A1CF,ENSG00000148584,UGAUCAGUAUA,MESNHKSGDGLSGTQKEAALRALVQRTGYSLVQENGQRKYGGPPPG...,"[[0.02515462040901184, -0.1693081259727478, 1....","[[0.102689214, -0.18220823, -0.05008613, 0.156..."
1,A1CF,ENSG00000148584,AUAAUUA,MESNHKSGDGLSGTQKEAALRALVQRTGYSLVQENGQRKYGGPPPG...,"[[-0.005481339991092682, -0.008752591907978058...","[[0.102689214, -0.18220823, -0.05008613, 0.156..."
2,A1CF,ENSG00000148584,UUAAUUA,MESNHKSGDGLSGTQKEAALRALVQRTGYSLVQENGQRKYGGPPPG...,"[[-0.006157027557492256, -0.04564734920859337,...","[[0.102689214, -0.18220823, -0.05008613, 0.156..."
3,A1CF,ENSG00000148584,AUAAUUG,MESNHKSGDGLSGTQKEAALRALVQRTGYSLVQENGQRKYGGPPPG...,"[[0.0016195997595787048, 0.0697508156299591, 1...","[[0.102689214, -0.18220823, -0.05008613, 0.156..."
4,A1CF,ENSG00000148584,UUAAUUG,MESNHKSGDGLSGTQKEAALRALVQRTGYSLVQENGQRKYGGPPPG...,"[[0.02757852151989937, 0.006868686527013779, 3...","[[0.102689214, -0.18220823, -0.05008613, 0.156..."
...,...,...,...,...,...,...
3247,SRSF1,ENSG00000136450,CGGCGGU,MSGGGVIRGPAGNNDCRIYVGNLPPDIRTKDIEDVFYKYGAIRDID...,"[[0.03172174096107483, -0.21967869997024536, -...","[[0.19127601, -0.11130471, 0.0977051, 0.116294..."
3248,SRSF2,ENSG00000161547,GAAAGGAGA,MSYGRPPPDVEGMTSLKVDNLTYRTSPDTLRRVFEKYGRVGDVYIP...,"[[0.013123728334903717, -0.012446027249097824,...","[[0.26023224, -0.14764462, 0.03352632, 0.18112..."
3249,HNRNPAB,ENSG00000197451,AUAGCA,MSEAGEEQPMETTGATENGHEAVPEASRGRGWTGAAAGAGGATAAP...,"[[-0.0034322012215852737, 0.000896155834197998...","[[0.019191047, 0.034460854, 0.13640618, 0.1387..."
3250,HNRNPA1,ENSG00000135486,UAGG,MSKSESPKEPEQLRKLFIGGLSFETTDESLRSHFEQWGTLTDCVVM...,"[[0.11391925066709518, -0.08793018758296967, 7...","[[0.14509255, -0.06513151, -0.03306822, 0.0129..."


In [ ]:
average_length = data['RBP_sequence'].apply(len).mean()
print(f"Average length of RBP_sequence: {average_length}")


Average length of RBP_sequence: 415.5270602706027


In [ ]:
# data = data[data['RBP_sequence'].apply(len) <= 600]


In [ ]:
average_length = data['RBP_sequence'].apply(len).mean()
print(f"Average length of RBP_sequence: {average_length}")


Average length of RBP_sequence: 415.5270602706027


In [ ]:
max_rbp_sequence_length = data['RBP_sequence'].apply(len).max()
max_motif_length = data['Motif'].apply(len).max()

print(f"Maximum RBP sequence length: {max_rbp_sequence_length}")
print(f"Maximum Motif length: {max_motif_length}")

# Use these lengths for padding
max_encoder_seq_length = max_motif_length
max_decoder_seq_length = max_rbp_sequence_length

Maximum RBP sequence length: 2542
Maximum Motif length: 12


In [ ]:
import pandas as pd
from scipy import stats

# Assuming meta4k is a DataFrame and 'sequence' is the column with sequence strings
# Calculate sequence lengths
data['sequence_length'] = data['RBP_sequence'].apply(len)

# Compute Z-scores of the lengths
data['z_scores'] = stats.zscore(data['sequence_length'])

# Filter the outliers; this will keep only the rows that are not outliers
filtered_data = data[(data['z_scores'] > -3) & (data['z_scores'] < 3)]

# Determine the outliers
outliers = data[(data['z_scores'] <= -3) | (data['z_scores'] >= 3)]

# Print the outliers
print(outliers)


     Gene_name          Gene_id         Motif  \
9         ADAR  ENSG00000160710          GCGC   
10        ADAR  ENSG00000160710         GCGCG   
16      ANKHD1  ENSG00000131503       AGACGAA   
17      ANKHD1  ENSG00000131503       AGACGUA   
18      ANKHD1  ENSG00000131503       AGACGAU   
19      ANKHD1  ENSG00000131503       AGACGUU   
117       DHX9  ENSG00000135829          CGCG   
118       DHX9  ENSG00000135829          GCGC   
119       DHX9  ENSG00000135829         GCGCG   
1409     PPRC1  ENSG00000148840       CCGCGCC   
1410     PPRC1  ENSG00000148840       GCGCGCC   
1411     PPRC1  ENSG00000148840       CGGCGCC   
1412     PPRC1  ENSG00000148840       GGGCGCC   
1413     PPRC1  ENSG00000148840       CCGCGCG   
1414     PPRC1  ENSG00000148840       GCGCGCG   
1415     PPRC1  ENSG00000148840       CGGCGCG   
1416     PPRC1  ENSG00000148840       GGGCGCG   
1539      PUM1  ENSG00000134644  AAUUGUACAUAA   
1540      PUM1  ENSG00000134644   CCAGAAUUGUA   
1541      PUM1  ENSG

In [ ]:
max_rbp_sequence_length = filtered_data['RBP_sequence'].apply(len).max()
min_rbp_sequence_length = filtered_data['RBP_sequence'].apply(len).min()
max_motif_length = filtered_data['Motif'].apply(len).max()
min_motif_length = filtered_data['Motif'].apply(len).min()

print(f"Maximum RBP sequence length: {max_rbp_sequence_length}")
print(f"Maximum Motif length: {max_motif_length}")
print(f"Minimum RBP sequence length: {min_rbp_sequence_length}")
print(f"Minimum Motif length: {min_motif_length}")

# Use these lengths for padding
max_encoder_seq_length = max_motif_length
max_decoder_seq_length = max_rbp_sequence_length

Maximum RBP sequence length: 1066
Maximum Motif length: 12
Minimum RBP sequence length: 136
Minimum Motif length: 4


In [ ]:
import numpy as np
import ast

### seq2seq model

In [ ]:
data.columns

Index(['Gene_name', 'Gene_id', 'Motif', 'RBP_sequence', 'rna_motif_emb',
       'rbp_esm_emb', 'sequence_length', 'z_scores'],
      dtype='object')

In [ ]:
print(data['rna_motif_emb'][1].shape)
print(data['rbp_esm_emb'][1].shape)

(7, 120)
(594, 1280)


In [ ]:
data = filtered_data.copy()

In [ ]:
data['sequence_length'].max()

1066

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Checking the GPU availability
if not torch.cuda.is_available():
    raise SystemError('GPU device not found')
print(f"Using device: {device}")


Using device: cuda


In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
!pip install fair-esm
##setting up ESM
import torch
import esm
esm_model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
esm_model.eval()  # disables dropout for deterministic results
esm_model.cuda() #push model to gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 2.9 MB/s eta 0:00:00


Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t33_650M_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t33_650M_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D-contact-regression.pt


ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0-32): 33 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=660, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((1280,), eps=1

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split

# 1. Dataset and DataLoader:
class Seq2SeqDataset(Dataset):
    def __init__(self, encoder_data, decoder_data, decoder_target):
        self.encoder_data = encoder_data
        self.decoder_data = decoder_data
        self.decoder_target = decoder_target

    def __len__(self):
        return len(self.encoder_data)

    def __getitem__(self, idx):
        return self.encoder_data[idx], self.decoder_data[idx], self.decoder_target[idx]

def pad_collate(batch):
    (encoder_seqs, decoder_seqs, decoder_targets) = zip(*batch)

    encoder_seqs_padded = torch.nn.utils.rnn.pad_sequence(encoder_seqs, batch_first=True, padding_value=0)
    decoder_seqs_padded = torch.nn.utils.rnn.pad_sequence(decoder_seqs, batch_first=True, padding_value=0)
    decoder_targets_padded = torch.nn.utils.rnn.pad_sequence(decoder_targets, batch_first=True, padding_value=0)

    return encoder_seqs_padded, decoder_seqs_padded, decoder_targets_padded

encoder_input_list = [torch.tensor(seq, dtype=torch.float32) for seq in data['rna_motif_emb'].tolist()]
decoder_input_list = [torch.tensor(seq, dtype=torch.float32) for seq in data['rbp_esm_emb'].tolist()]
decoder_target_list = [torch.tensor(np.roll(seq, shift=-1, axis=0), dtype=torch.float32) for seq in data['rbp_esm_emb'].tolist()]

dataset = Seq2SeqDataset(encoder_input_list, decoder_input_list, decoder_target_list)

# Splitting the dataset into training and test/validation sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=pad_collate)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=pad_collate)

# 2. Model Definition and Training Loop:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments

# The AdapterNetwork is the same as the previous code
class AdapterNetwork(torch.nn.Module):
    def __init__(self, output_size=1024):  # Remove the input_size from the initialization
        super(AdapterNetwork, self).__init__()
        self.fc = None  # Initialize the Linear layer as None
        self.output_size = output_size

    def forward(self, input):
        # Check if the Linear layer has been initialized
        if self.fc is None:
            # Dynamically compute the input size based on the shape of the input tensor
            input_size = input.shape[-1]
            self.fc = torch.nn.Linear(input_size, self.output_size).to(input.device)  # Ensure it's on the same device as the input

        return self.fc(input)

class CustomT5Model(T5ForConditionalGeneration):
    def __init__(self, config, adapter_network):
        super(CustomT5Model, self).__init__(config)
        self.adapter_network = adapter_network

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        decoder_input_ids=None,
        decoder_attention_mask=None,
        rna_motif_emb=None,
        **kwargs
    ):
        # Get the encoder outputs
        encoder_outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )

        # Here, we are using RNA motif embeddings directly instead of DNAModel
        print(encoder_outputs[0].shape)
        print(rna_motif_emb.shape)
        joint_embedding = self.adapter_network(torch.cat((encoder_outputs[0], rna_motif_emb), dim=-1))

        # Get the decoder outputs
        decoder_outputs = self.decoder(
            input_ids=decoder_input_ids,
            attention_mask=decoder_attention_mask,
            encoder_hidden_states=joint_embedding,
            **kwargs
        )

        lm_head_outputs = self.lm_head(decoder_outputs[0])

        return lm_head_outputs

# Initialize the adapter network
# adapter_input_size = 120 + 1024  # Assuming the size of T5 encoder outputs is 1024
# adapter_output_size = 1024  # The expected size for T5 decoder inputs
# adapter_network = AdapterNetwork(adapter_input_size, adapter_output_size)


adapter_output_size = 1024  # The expected size for T5 decoder inputs
adapter_network = AdapterNetwork(adapter_output_size)


# Initialize the custom T5 model
model = CustomT5Model(config=T5ForConditionalGeneration.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc').config, adapter_network=adapter_network)

# Now, you can use the model in the usual way with the provided train_dataloader and test_dataloader for training and evaluation.


config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.42G [00:00<?, ?B/s]

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at Rostlab/prot_t5_xl_half_uniref50-enc and are newly initialized: ['decoder.block.20.layer.1.EncDecAttention.o.weight', 'decoder.block.11.layer.1.EncDecAttention.v.weight', 'decoder.block.14.layer.1.EncDecAttention.v.weight', 'decoder.block.6.layer.2.layer_norm.weight', 'decoder.block.10.layer.2.layer_norm.weight', 'decoder.block.17.layer.0.SelfAttention.v.weight', 'decoder.block.20.layer.0.SelfAttention.o.weight', 'decoder.block.14.layer.0.SelfAttention.q.weight', 'decoder.block.21.layer.0.layer_norm.weight', 'decoder.block.1.layer.2.DenseReluDense.wi.weight', 'decoder.block.21.layer.2.DenseReluDense.wi.weight', 'decoder.block.6.layer.0.SelfAttention.v.weight', 'decoder.block.18.layer.1.EncDecAttention.o.weight', 'decoder.block.21.layer.1.EncDecAttention.o.weight', 'decoder.block.5.layer.1.EncDecAttention.o.weight', 'decoder.block.18.layer.2.layer_norm.weight', 'decoder.block.11.layer.1.EncDecAt

In [ ]:
encoder_input_list[0].shape

torch.Size([11, 120])

In [ ]:
train_dataset = train_dataloader.dataset

In [ ]:
len(train_dataset[1])

3

In [ ]:
rna_motif_emb[11].shape

(5, 120)

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

In [ ]:
# Loss function
criterion = torch.nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Number of training epochs
n_epochs = 3

# Move the model to the appropriate device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Lists to store losses for plotting
train_losses = []
test_losses = []

for epoch in range(n_epochs):
    print(f"Epoch {epoch + 1}/{n_epochs}")

    # Training
    model.train()
    total_loss = 0
    current_index = 0

    for encoder_data, decoder_input, decoder_target in train_dataloader:
        batch_size = encoder_data.size(0)
        batch_rna_motif_emb = rna_motif_emb[current_index:current_index + batch_size]
        current_index += batch_size

        max_length = max(len(item) for item in batch_rna_motif_emb)
        padded_rna_motif_emb = np.array(
            [np.pad(item, ((0, max_length - len(item)), (0, 0)), 'constant', constant_values=0)
             for item in batch_rna_motif_emb],
            dtype=np.float32
        )

        # Move data to device
        encoder_data = encoder_data.to(device)
        decoder_input = decoder_input.to(device)
        decoder_target = decoder_target.to(device)

        # Prepare rna_motif_emb_tensor and ensure it's on the same device as encoder_data
        rna_motif_emb_tensor = torch.from_numpy(padded_rna_motif_emb).to(device)

        joint_embedding = torch.cat((encoder_data, rna_motif_emb_tensor), dim=1)
        print(joint_embedding.shape)
        joint_embedding_flattened = joint_embedding.view(joint_embedding.size(0), -1)
        print(joint_embedding_flattened.shape)
        encoder_data_transformed = adapter_network(joint_embedding_flattened)
        print(encoder_data_transformed)
        outputs = model(inputs_embeds=encoder_data_transformed,
                        attention_mask=None,
                        decoder_input_ids=decoder_input)

        loss = criterion(outputs.view(-1, outputs.size(-1)), decoder_target.view(-1))
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    average_train_loss = total_loss / len(train_dataloader)
    train_losses.append(average_train_loss)
    print(f"Training loss: {average_train_loss}")

    # Evaluation on test data
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for encoder_data, decoder_input, decoder_target in test_dataloader:
            encoder_data = encoder_data.to(device)
            decoder_input = decoder_input.to(device)
            decoder_target = decoder_target.to(device)

            encoder_data_transformed = adapter_network(encoder_data)
            outputs = model(input_ids=encoder_data_transformed,
                            attention_mask=None,
                            decoder_input_ids=decoder_input)

            loss = criterion(outputs.view(-1, outputs.size(-1)), decoder_target.view(-1))
            total_loss += loss.item()

    average_test_loss = total_loss / len(test_dataloader)
    test_losses.append(average_test_loss)
    print(f"Validation loss: {average_test_loss}")

# Plot the training and validation losses
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(test_losses, label='Validation Loss')
plt.title('Training and Validation Losses over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Epoch 1/3
torch.Size([4, 20, 120])
torch.Size([4, 2400])
tensor([[ 0.0523, -0.1181,  0.0028,  ...,  0.0648, -0.0117, -0.0824],
        [-0.0317, -0.0724, -0.0091,  ...,  0.0167,  0.0228,  0.0039],
        [-0.0215, -0.0459,  0.0064,  ...,  0.0806,  0.0582, -0.0232],
        [-0.0764, -0.0665, -0.0339,  ...,  0.0605, -0.0028, -0.0394]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


ValueError: ignored

In [ ]:
encoder_data_transformed.shape

In [ ]:
encoder_data.shape

In [ ]:
 rna_motif_emb_tensor.shape

In [ ]:
joint_embedding.shape

In [ ]:
rna_motif_emb.shape